In [1]:
# Standard libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import librosa
from google.colab import drive
import os

from librosa import load as lload
from librosa.feature import mfcc
from librosa.feature.inverse import mfcc_to_audio

# Mount Google Drive for access
drive.mount('/content/drive')

from sklearn.preprocessing import MinMaxScaler

# Base directory path
base_directory_path = '/content/drive/MyDrive/207-Project/notebooks/RG/3_species/librosa_loaded_sr16000/'



Mounted at /content/drive


In [2]:

# Function to extract 5-second  chunks from audio
def extract_5sec_chunks(
    audio_array: np.ndarray,
    window_size_s: float = 5.0,
    hop_size_s: float = 2.5, #(with overlap)
    sample_rate=16000,
    # n_fft=2048,
    # n_mfcc=13,
) -> np.ndarray:
    frame_length = int(window_size_s * sample_rate)
    frame_step = int(hop_size_s * sample_rate)
    framed_audio = tf.signal.frame(audio_array, frame_length, frame_step, pad_end=False)

    return framed_audio



In [3]:


# Load metadata
dataset_path = '/content/drive/MyDrive/207-Project/notebooks/RG/3_species/'
metadata_path = os.path.join(dataset_path, "train_val.csv")
metadata_df = pd.read_csv(metadata_path)

# Create a dictionary to map filenames to labels
labels_dict = metadata_df.set_index('filename_npy')['primary_label'].to_dict()

# Get unique filenames from the metadata
filenames = set(base_directory_path + '/' + metadata_df['filename_npy'].unique())

# Split train and validation data
train_df = metadata_df[metadata_df['data'] == 'train']
validate_df = metadata_df[metadata_df['data'] == 'val']

In [4]:
def mfcc_chunks(dataframe: pd.DataFrame, sample_rate=16000, n_mfcc=20) -> tuple[np.ndarray, np.ndarray]:
    y = []
    X = []

    for i, row in dataframe.iterrows():
      label = row['primary_label']
      tensor_5sec = row['audio_chunks']

      for each in tensor_5sec:
        y.append(label)
        each = np.array(each)
        X_mfcc = mfcc(y=each, sr=sample_rate)
        X.append(X_mfcc)

    assert len(y) == len(X)

    y = np.array(y)
    X = np.array(X)



    return X, y

In [5]:
# Load all train audio data one time
train_audios = []
for filename in train_df['filename_npy']:
    audio = np.load('/content/drive/MyDrive/207-Project/data/train/librosa_loaded/' + filename)
    train_audios.append(audio)

In [6]:
# Load all validate audio data one time
val_audios = []
for filename in validate_df['filename_npy']:
    audio = np.load('/content/drive/MyDrive/207-Project/data/train/librosa_loaded/' + filename)
    val_audios.append(audio)

In [7]:
print(len(train_audios))
print(len(val_audios))

657
283


In [8]:
#Initialize an empty list to store the chunk data for training data
train_chunks = []

for audio in train_audios:
  train_chunks.append(extract_5sec_chunks(audio))

len(train_chunks)

657

In [9]:
#Initialize an empty list to store the chunk data for VALIDATION data
val_chunks = []

for audio in val_audios:
  val_chunks.append(extract_5sec_chunks(audio))

len(val_chunks)

283

In [10]:
#add chunks to trainig data
train_df['audio_chunks'] = train_chunks
train_df.head()
train_df = train_df.sample(frac=1, random_state=1234)




<ipython-input-10-3809d4e98fc2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['audio_chunks'] = train_chunks


In [11]:
#add chunks to validation data
validate_df['audio_chunks'] = val_chunks
validate_df.head()
validate_df = validate_df.sample(frac=1, random_state=1234)

<ipython-input-11-0cb3c619c1e6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  validate_df['audio_chunks'] = val_chunks


In [12]:
%who


LabelEncoder	 MinMaxScaler	 StandardScaler	 audio	 base_directory_path	 dataset_path	 drive	 extract_5sec_chunks	 filename	 
filenames	 labels_dict	 layers	 librosa	 lload	 metadata_df	 metadata_path	 mfcc	 mfcc_chunks	 
mfcc_to_audio	 models	 np	 os	 pd	 tf	 train_audios	 train_chunks	 train_df	 
train_test_split	 val_audios	 val_chunks	 validate_df	 


In [13]:
del audio
del base_directory_path
del dataset_path
del drive
del extract_5sec_chunks
del filename
del filenames
# del labels_dict
del layers
del metadata_df
del metadata_path
del os
del train_audios
del train_chunks
del val_audios
del val_chunks
del train_test_split


In [14]:
import gc
gc.collect()

27198

In [15]:
%who


LabelEncoder	 MinMaxScaler	 StandardScaler	 gc	 labels_dict	 librosa	 lload	 mfcc	 mfcc_chunks	 
mfcc_to_audio	 models	 np	 pd	 tf	 train_df	 validate_df	 


In [16]:
#convert to X and y
X_train, y_train = mfcc_chunks(train_df)

In [17]:
#convert to X and y
X_val, y_val = mfcc_chunks(validate_df)

In [18]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)

X_train shape: (7889, 20, 157)
y_train shape: (7889,)
X_val shape: (3121, 20, 157)
y_val shape: (3121,)


In [19]:
# validate_df.head()
print(train_df['primary_label'].unique())

['comsan' 'eaywag1' 'barswa']


In [20]:
del train_df
del validate_df
del mfcc
del mfcc_chunks
del mfcc_to_audio

In [21]:
import json

In [22]:
X_train_list = X_train.tolist()
y_train_list = y_train.tolist()

X_val_list = X_val.tolist()
y_val_list = y_val.tolist()

# Save data to JSON files
with open('/content/drive/MyDrive/My-207/X_train_mfcc.json', 'w') as file:
    json.dump(X_train_list, file)

with open('/content/drive/MyDrive/My-207/y_train_mfcc.json', 'w') as file:
    json.dump(y_train_list, file)

with open('/content/drive/MyDrive/My-207/X_val_mfcc.json', 'w') as file:
    json.dump(X_val_list, file)

with open('/content/drive/MyDrive/My-207/y_val_mfcc.json', 'w') as file:
    json.dump(y_val_list, file)